In [1]:
import os
proxy = 'http://10.2.176.162:8080'
os.environ['http_proxy'] = proxy 
os.environ['HTTP_PROXY'] = proxy
os.environ['https_proxy'] = proxy
os.environ['HTTPS_PROXY'] = proxy

In [2]:
import os
from datetime import datetime, timedelta, date
from pytz import timezone
import urllib3
import ssl
import json
import pandas as pd
from openapi_client import openapi

import configparser
c = configparser.ConfigParser()
c.read('setup.cfg')
token = c['tinkoff']['token']

if (not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None)):
    ssl._create_default_https_context = ssl._create_unverified_context
    
http = urllib3.proxy_from_url('http://10.2.176.162:8080')

In [3]:
#подключение к API
# client = openapi.sandbox_api_client(token)
client = openapi.api_client(token)
client

#### Содержимое портфеля

In [5]:
pf = client.portfolio.portfolio_get()

MaxRetryError: HTTPSConnectionPool(host='api-invest.tinkoff.ru', port=443): Max retries exceeded with url: /openapi//portfolio (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001B056D68940>: Failed to establish a new connection: [WinError 10060] Попытка установить соединение была безуспешной, т.к. от другого компьютера за требуемое время не получен нужный отклик, или было разорвано уже установленное соединение из-за неверного отклика уже подключенного компьютера'))

In [5]:
pf = client.portfolio.portfolio_get()
pf_columns = [
    'figi',
    'ticker',
    'isin',
    'instrument_type',
    'balance',
    'blocked',
    'expected yield currency',
    'expected yield value',
    'lots',
    'avg pos price currency',
    'avg pos price value',
    'avg pos price no_nkd',
    'name',
]

df = pd.DataFrame(columns = pf_columns)

for pos in pf.payload.positions:
    df = df.append(
            pd.Series(
                [
                    pos.figi,
                    pos.ticker,
                    pos.isin,
                    pos.instrument_type,
                    pos.balance,
                    pos.blocked,
                    pos.expected_yield.currency,
                    pos.expected_yield.value,
                    pos.lots,
                    pos.average_position_price.currency,
                    pos.average_position_price.value,
                    pos.average_position_price_no_nkd,
                    pos.name,
                ],
            index = pf_columns),
        ignore_index = True
        )
df.head()

KeyboardInterrupt: 

In [38]:
pf_dict = pf.to_dict()
usd_cost = pf_dict['USD000UTSTOM']
# pf_dict['average_position_price']['value']

{'tracking_id': 'b432132272700041',
 'status': 'Ok',
 'payload': {'positions': [{'figi': 'BBG000CL9VN6',
    'ticker': 'NFLX',
    'isin': 'US64110L1061',
    'instrument_type': 'Stock',
    'balance': 2.0,
    'blocked': None,
    'expected_yield': {'currency': 'USD', 'value': 28.2},
    'lots': 2,
    'average_position_price': {'currency': 'USD', 'value': 502.9},
    'average_position_price_no_nkd': None,
    'name': 'Netflix'},
   {'figi': 'BBG000BVV7G1',
    'ticker': 'TXN',
    'isin': 'US8825081040',
    'instrument_type': 'Stock',
    'balance': 4.0,
    'blocked': None,
    'expected_yield': {'currency': 'USD', 'value': 30.2},
    'lots': 4,
    'average_position_price': {'currency': 'USD', 'value': 158.67},
    'average_position_price_no_nkd': None,
    'name': 'Texas Instruments'},
   {'figi': 'BBG000QXWHD1',
    'ticker': 'BIDU',
    'isin': 'US0567521085',
    'instrument_type': 'Stock',
    'balance': 3.0,
    'blocked': None,
    'expected_yield': {'currency': 'USD', 'val

In [51]:
for pos in pf.payload.positions:
    if pos.ticker == 'USD000UTSTOM':
        usd_value = pos.average_position_price.value
    else:
        usd_value = 100

pf_amount_rub = 0
for pos in pf.payload.positions:
    rub_price = ((pos.average_position_price.value * usd_value)
#                  if pos.average_position_price.currency == 'USD'
#                  else pos.average_position_price.value
    )
    rub_yield = pos.expected_yield.value * usd_value
    pf_price_rub = pf_amount_rub + rub_value
    pf_yield_rub = pf_price_rub + rub_yield
    print(pos.name,
          pos.average_position_price.currency,
          pos.average_position_price.value,
          rub_value,
          pf_price_rub,
          rub_yield,
          pf_yield_rub
         )

Netflix USD 502.9 5777.520100000001 5777.520100000001 2143.482 7921.002100000001
Texas Instruments USD 158.67 5777.520100000001 5777.520100000001 2295.502 8073.0221
Baidu USD 144.32 5777.520100000001 5777.520100000001 -50.926700000000004 5726.593400000001
D.R. Horton USD 76.18 5777.520100000001 5777.520100000001 -1190.3166 4587.2035000000005
Walt Disney USD 147.82 5777.520100000001 5777.520100000001 2753.8423 8531.3624
Salesforce USD 227.49 5777.520100000001 5777.520100000001 145.9392 5923.4593
Adobe USD 487.44 5777.520100000001 5777.520100000001 362.5677 6140.0878
IDEXX USD 456.15 5777.520100000001 5777.520100000001 180.90380000000002 5958.423900000001
Motorola Solutions USD 172.34 5777.520100000001 5777.520100000001 -908.3195000000001 4869.2006
AMD USD 86.95 5777.520100000001 5777.520100000001 1042.8572000000001 6820.377300000001
Fate Therapeutics Inc USD 55.8 5777.520100000001 5777.520100000001 10337.36 16114.880100000002
AECOM USD 52.6 5777.520100000001 5777.520100000001 -495.5852 

In [52]:
client.market.market_search_by_figi_get('BBG0013HGFT4')

MaxRetryError: HTTPSConnectionPool(host='api-invest.tinkoff.ru', port=443): Max retries exceeded with url: /openapi//market/search/by-figi?figi=BBG0013HGFT4 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001EF7B827BB0>: Failed to establish a new connection: [WinError 10060] Попытка установить соединение была безуспешной, т.к. от другого компьютера за требуемое время не получен нужный отклик, или было разорвано уже установленное соединение из-за неверного отклика уже подключенного компьютера'))

# Атрибуты для каждой позиции в портфеле

In [27]:
for pos in pf.payload.positions:
    if pos.name == 'Доллар США': 
        for attr in pos.attribute_map:
            print(f'{attr}:', getattr(pos, attr))
        break

figi: BBG0013HGFT4
ticker: USD000UTSTOM
isin: None
instrument_type: Currency
balance: 71.29
blocked: None
expected_yield: {'currency': 'RUB', 'value': -202.82}
lots: 0
average_position_price: {'currency': 'RUB', 'value': 76.01}
average_position_price_no_nkd: None
name: Доллар США


#### Получить подневную историю инструмента по тикеру

In [16]:
client.market.market_candles_get(
        'BBG005VKB7D7', 
        datetime.now(tz=timezone('Europe/Moscow')) - timedelta(days=1), 
        datetime.now(tz=timezone('Europe/Moscow')), 
        'day')

{'payload': {'candles': [{'c': 3912.0,
                          'figi': 'BBG005VKB7D7',
                          'h': 3935.5,
                          'interval': 'day',
                          'l': 3904.0,
                          'o': 3930.0,
                          'time': datetime.datetime(2020, 12, 8, 7, 0, tzinfo=tzutc()),
                          'v': 7719}],
             'figi': 'BBG005VKB7D7',
             'interval': 'day'},
 'status': 'Ok',
 'tracking_id': '5162f18f62a21beb'}

#### Получить подневную историю инструмента по тикеру (датафрейм)

In [12]:
def get_daily_history(ticker: str, date_from: str, date_to: str, interval_in='day') -> pd.DataFrame:
    my_columns = ['date','ticker','open','highest','lowest','close','volume']
    df = pd.DataFrame(columns = my_columns)

    dt_from = datetime.strptime(date_from, '%Y-%m-%d')
    dt_from_with_tz = dt_from.replace(tzinfo=timezone('Europe/Moscow'))

    dt_to = datetime.strptime(date_to, '%Y-%m-%d')
    dt_to_with_tz = dt_to.replace(tzinfo=timezone('Europe/Moscow'))

    figi = get_figi_by_ticker(ticker)
    hisory = client.market.market_candles_get(figi, dt_from_with_tz, dt_to_with_tz, interval_in)
    
    for candle in hisory.payload.candles:
        df = df.append(
            pd.Series(
                [
                    candle.time,
                    ticker,
                    candle.o,
                    candle.h,
                    candle.l,
                    candle.c,
                    candle.v
                   
                ],
            index = my_columns),
        ignore_index = True
        )
    return df
get_daily_history('FXCN', '2020-12-01', '2020-12-07')

,date,ticker,open,highest,lowest,close,volume
0,2020-12-01 07:00:00+00:00,FXCN,4136.0,4149.0,4060.5,4068.0,34983
1,2020-12-02 07:00:00+00:00,FXCN,4100.0,4106.5,3988.0,4002.5,40515
2,2020-12-03 07:00:00+00:00,FXCN,4030.0,4049.0,3990.0,4011.5,22660
3,2020-12-04 07:00:00+00:00,FXCN,4012.0,4047.5,3942.0,3990.0,25765


#### Получить портфель по тикерам (датафрейм)

In [19]:
current_date = datetime.now(tz=timezone('Europe/Moscow')) - timedelta(days=1)
union_df = pd.DataFrame()
for pos in pf.payload.positions:
    df = get_daily_history(pos.ticker, '2020-12-07', '2020-12-08')
    df['currency'] = pos.average_position_price.currency
    df['name'] = pos.name
    df['lots'] = pos.lots
    df['cur_value'] = df['lots'] * df['close']
    df['rub_value'] = df.apply(lambda row: row['cur_value'] * 76 if row['currency'] == 'USD' else row['cur_value'], axis=1)
    union_df = pd.concat([union_df, df])
union_df.head()

,date,ticker,open,highest,lowest,close,volume,currency,name,lots,cur_value,rub_value
0,2020-12-07 07:00:00+00:00,NFLX,499.96,517.71,496.57,515.78,1175365,USD,Netflix,2,1031.56,78398.56
0,2020-12-07 07:00:00+00:00,TXN,165.81,167.91,165.48,166.29,2024111,USD,Texas Instruments,4,665.16,50552.16
0,2020-12-07 07:00:00+00:00,BIDU,145.00,145.14,141.50,144.30,690969,USD,Baidu,3,432.90,32900.40
0,2020-12-07 07:00:00+00:00,DHI,71.55,73.99,70.65,73.80,431798,USD,D.R. Horton,6,442.80,33652.80
0,2020-12-07 07:00:00+00:00,DIS,153.27,155.23,152.29,153.68,723313,USD,Walt Disney,7,1075.76,81757.76


In [23]:
union_df[union_df.name == 'Доллар США']

,date,ticker,open,highest,lowest,close,volume,currency,name,lots,cur_value,rub_value
0,2020-12-07 07:00:00+00:00,USD000UTSTOM,74.2675,74.52,73.3,73.64,3863550,RUB,Доллар США,0,0.0,0.0


In [28]:
import plotly.express as px
fig = px.pie(
        union_df,
        values='rub_value',
        names='name',
        labels='rub_value'
#         color_discrete_sequence=['#fbec5d', '#099f77', '#ff4040', '#808080', '#ffffff', '#1ab5d0', '#fa6600', '#dda0dd'],
#         category_orders={"operator_name": ["BEE", "MGF", "MTS", "Other","Tele2","YOTA","Мотив","Летай"]},
#         width=1000,
#         height=50,
#         line_shape='spline' #linear
        )
fig.show()

In [84]:
# etfs = client.market.market_etfs_get() 
# stocks = client.market.market_stocks_get() 
# instr_list = etfs.payload.instruments + stocks.payload.instruments
# instr_list

In [33]:
import openapi_client.openapi_streaming
help(openapi_client.openapi_streaming)

Help on module openapi_client.openapi_streaming in openapi_client:

NAME
    openapi_client.openapi_streaming

FUNCTIONS
    callback_decider(event_string, on_candle_event, on_orderbook_event, on_instrument_info_event)
    
    do_nothing(event)
    
    print_ = print(...)
        print(value, ..., sep=' ', end='\n', file=sys.stdout, flush=False)
        
        Prints the values to a stream, or to sys.stdout by default.
        Optional keyword arguments:
        file:  a file-like object (stream); defaults to the current sys.stdout.
        sep:   string inserted between values, default a space.
        end:   string appended after the last value, default a newline.
        flush: whether to forcibly flush the stream.
    
    print_event(event)
    
    run_stream_consumer(token, candle_subs=(), orderbook_subs=(), instrument_info_subs=(), on_candle_event=<function do_nothing at 0x000001EF7B7C1AF0>, on_orderbook_event=<function do_nothing at 0x000001EF7B7C1AF0>, on_instrument_info_

In [1]:
from openapi_client.openapi_streaming import run_stream_consumer
from openapi_client.openapi_streaming import print_event

In [2]:
help(run_stream_consumer)

Help on function run_stream_consumer in module openapi_client.openapi_streaming:

run_stream_consumer(token, candle_subs=(), orderbook_subs=(), instrument_info_subs=(), on_candle_event=<function do_nothing at 0x00000218416A6820>, on_orderbook_event=<function do_nothing at 0x00000218416A6820>, on_instrument_info_event=<function do_nothing at 0x00000218416A6820>)
    | subscriptions example:
    |    candle_subs = [{'figi': 'BBG000B9XRY4', 'interval': '1min'}, {'figi': 'BBG009S39JX6', 'interval': '1min'}]
    |    orderbook_subs = [{'figi': 'BBG000B9XRY4', 'depth': 3}, {'figi': 'BBG009S39JX6', 'depth': 3}]
    |    instrument_info_subs = [{'figi': 'BBG000B9XRY4'}, {'figi': 'BBG009S39JX6'}]
    |
    | callback example:
    |    lambda x: print(x)



In [5]:
candle_subs = [{'figi': 'BBG000B9XRY4', 'interval': '1min'}, {'figi': 'BBG009S39JX6', 'interval': '1min'}]
run_stream_consumer(token,
                    candle_subs,
                    on_candle_event=print_event,
                    on_orderbook_event=print_event,
                    on_instrument_info_event=print_event)
lambda x: print(x)

[WinError 10060] Попытка установить соединение была безуспешной, т.к. от другого компьютера за требуемое время не получен нужный отклик, или было разорвано уже установленное соединение из-за неверного отклика уже подключенного компьютера


<function __main__.<lambda>(x)>

In [31]:


candle_subs = [{'figi': 'BBG000B9XRY4', 'interval': '1min'}, {'figi': 'BBG009S39JX6', 'interval': '1min'}]
orderbook_subs = [{'figi': 'BBG0013HGFT4', 'depth': 1}, {'figi': 'BBG009S39JX6', 'depth': 3}]
instrument_info_subs = [{'figi': 'BBG000B9XRY4'}, {'figi': 'BBG009S39JX6'}]

run_stream_consumer(token,
                    candle_subs, orderbook_subs, instrument_info_subs,
                    on_candle_event=print_event,
                    on_orderbook_event=print_event,
                    on_instrument_info_event=print_event)

b'\x92\xbd\xa3\x1b\x8a\x8b^\x00\x93\xe3\xdc\x1a\r\xf7V\xfe'
b'\x92\xbd\xa3\x1b\x8a\x8b^\x00\x93\xe3\xdc\x1a\r\xf7V\xfe'
b'\xa2\x04\xb0\xca\xb2\x95\xaf\xd9 E\t\xfcH\xe1f\xbd'
b'\xa2\x04\xb0\xca\xb2\x95\xaf\xd9 E\t\xfcH\xe1f\xbd'
b'\xe4\xe0h\x02%#=\xc4\xc8Zm_Y\x97\n\x1a\xd8\x0b*\xf7'
b'\xe4\xe0h\x02%#=\xc4\xc8Zm_Y\x97\n\x1a\xd8\x0b*\xf7'
sending: { "event": "candle:subscribe", "figi": "BBG000B9XRY4", "interval": "1min" }
sending: { "event": "candle:subscribe", "figi": "BBG009S39JX6", "interval": "1min" }
sending: { "event": "orderbook:subscribe", "figi": "BBG0013HGFT4", "depth": 1 }
sending: { "event": "orderbook:subscribe", "figi": "BBG009S39JX6", "depth": 3 }
sending: { "event": "instrument_info:subscribe", "figi": "BBG000B9XRY4" }
sending: { "event": "instrument_info:subscribe", "figi": "BBG009S39JX6" }
{'o': 123.58, 'c': 123.58, 'h': 123.58, 'l': 123.58, 'v': 49, 'time': '2020-12-08T10:49:00Z', 'interval': '1min', 'figi': 'BBG000B9XRY4'}
{'o': 1814, 'c': 1814, 'h': 1814, 'l': 1814, '